In [ ]:
import csv
import numpy as np
from numpy.random import permutation
import keras 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.regularizers import l2, activity_l2
from sklearn.model_selection import KFold
%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True)   

In [ ]:
# normalize_data cannot get better result
def read_data():
    features=[]   
    print("read train data")
    with open ('data/train.csv') as train_csv:
        train_csv.readline()
        for row in csv.reader(train_csv, delimiter=','):
            if row[2] == 'PM2.5':
                tmp = row[3:]
                features.append(row[3:])             
    X_array = np.array(features, dtype=np.float32)  
    return X_array

In [ ]:
 def read_test_data():
    X_train=[]
    print("read test data")
    with open ('data/test_X.csv') as train_csv:
        for row in csv.reader(train_csv, delimiter=','):
            if row[1] == 'PM2.5': 
                tmp = row[2:]         
                X_train.append(tmp)        

    X_array = np.array(X_train, dtype=np.float32)
    ones = np.ones((X_array.shape[0], 1), dtype=np.int32)
    X_array = np.append(X_array, ones, axis=1)
    return X_array

In [ ]:
# expand continue hours of continue day into one training data
def expand_data(X_array):
    #print X_array.shape
    x_month = X_array.reshape(12, X_array.shape[0]/12 * X_array.shape[1])
    collect_X_train = np.zeros((x_month.shape[0] * (x_month.shape[1]-9),10), dtype=np.float32)
    collect_y_train = np.zeros((x_month.shape[0] * (x_month.shape[1]-9) , 1), dtype=np.float32)
    
    # some raw data value is -1, align it with before hours result
    for i in range(x_month.shape[0]):
        for j in range (x_month.shape[1]):
            if (j>1 and x_month[i,j]<0):
                x_month[i,j] = x_month[i,j-1]
    #print 'collect_X_train=', collect_X_train.shape
    #print 'x_month=', x_month.shape
    test=0
    for idx in range(x_month.shape[0]):
        for i in range(x_month.shape[1]-9):
            collect_X_train[test, 0:9] = x_month[idx, i:i+9].ravel()
            collect_X_train[test, 9] = 1
            collect_y_train[test] = x_month[idx, i+9]
            #print collect_X_train[test], collect_y_train[test]
            test+=1           
    return collect_X_train, collect_y_train

#X_data, X_mean, X_scale = read_data()
#yyy,  zzz= expand_data(X_data)    
#print yyy.shape
#print yyy[-1]

In [ ]:
raw_data = read_data()
X_train, y_train = expand_data(raw_data)

n_samples = X_train.shape[0]
n_features = X_train.shape[1]

# Parameters
learning_rate = 0.001
training_epochs = 10000
display_step = 2000
reg = 0.001

# tf Graph Input
X = tf.placeholder(tf.float32, [n_samples, n_features])
Y = tf.placeholder(tf.float32, [n_samples, 1])

# weights
W = tf.Variable(tf.zeros([n_features, 1], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1], dtype=np.float32), name="bias")

# Construct a linear model
pred = tf.add(tf.matmul(X, W), b)

regularizer = tf.reduce_sum(tf.square(W)) /(2*n_samples)
# Mean squared error
cost = tf.reduce_sum(tf.square(pred-Y))/(2*n_samples)
loss = cost + reg * regularizer

# Gradient descent
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
final_weight = []
final_bias= []
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        sess.run(optimizer, feed_dict={X: np.asarray(X_train), Y: np.asarray(y_train)})

        #Display logs per epoch step
        if (epoch+1) % display_step == 0:
            c = sess.run(loss, feed_dict={X: np.asarray(X_train), Y: np.asarray(y_train)})
            print "Epoch:", '%04d' % (epoch+1), "loss=", "{:.9f}".format(c)             

    print "Optimization Finished!"
    training_loss = sess.run(loss, feed_dict={X: np.asarray(X_train), Y: np.asarray(y_train)})
    print "Training loss=", training_loss , '\n'  
    final_weight = sess.run(W)
    final_bias = sess.run(b)

In [ ]:
X_predict = read_test_data()
import pandas as pd
predictions = np.dot(X_predict, final_weight) + final_bias
index= []
for i in range(len(predictions)):
    index.append('id_'+ str(i))  
idx_array = np.array(index, dtype=np.str) 
#print idx_array.shape
#print predictions.shape

submission = pd.DataFrame({'id' : idx_array, 'value': predictions.flatten()})
submission.to_csv('test_result.csv',index=False)
# use tensorlfow implementation

In [ ]:
# use my implementation
def cost_func(X, y, weight, reg):
    scores =  X.dot(weight) 
    diff = scores - y
    loss =  np.sum(pow(diff, 2))
    weight0 = weight
    weight0[-1] = 0
    regular =  reg * np.sum(weight0 * weight0) 
    return 0.5 * (loss + regular) / y.shape[0]
    
def  cost_grad_func(X, y, weight, reg):
    num_train = y.shape[0]
    num_features = X.shape[1]
    grad = np.zeros((num_features, 1)) 
    scores =  X.dot(weight) 
    diff = scores - y
    cost = cost_func(X, y, weight, reg)
    weight0 = weight
    weight0[-1] = 0
    grad = (X.T.dot(diff)+ reg * weight0) / num_train
    return cost, grad

In [ ]:
raw_data = read_data()
X_train, y_train = expand_data(raw_data)

n_samples = X_train.shape[0]
n_features = X_train.shape[1]
learning_rate=0.0001
reg=1e-5
num_iters=20000
batch_size=100
loss_history = []
test_loss_history = []
weight = 0.001 * np.random.randn(n_features, 1).astype(np.float)

print "weight.shape=", weight.shape
print "X_train.shape=", X_train.shape

# train
for it in xrange(num_iters):
    X_batch = None
    y_batch = None
    batch_index = np.random.choice(n_samples, batch_size)
    X_batch = X_train[batch_index]
    y_batch = y_train[batch_index]
    loss, grad = cost_grad_func(X_batch, y_batch, weight, reg)  
    weight = weight - learning_rate * grad
    if it % 1000 == 0:
        print 'iteration %d / %d: loss %f' % (it, num_iters, loss)
        loss_history.append(loss)

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_history)
plt.title('loss_history')
plt.ylabel('loss')
plt.xlabel('iterator')
plt.legend(['loss'], loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
X_predict = read_test_data()

import pandas as pd
predictions = np.dot(X_predict, weight)
index= []
for i in range(len(predictions)):
    index.append('id_'+ str(i))  
idx_array = np.array(index, dtype=np.str) 

submission = pd.DataFrame({'id' : idx_array, 'value': predictions.flatten()})
submission.to_csv('test_result.csv',index=False)
# use my implementation